In [142]:
from qoop.core import state, ansatz, metric
import qiskit
from qiskit import transpile
from qiskit.quantum_info import Operator, DensityMatrix, Kraus
from scipy.linalg import qr
import numpy as np
import tensorflow as tf

In [143]:
# Step 1: Read quantum compilation and learn how to use qoop
# https://github.com/vutuanhai237/qoop/wiki/Advance:-Custom-state-preparation

# Step 2: Implement the following function

def V(num_qubits: int):
    '''
        Create a ansatz V with n qubits and assign its parameters
    '''
    #Assign random parameter
    circuit = ansatz.stargraph(num_qubits=num_qubits)
    num_params = circuit.num_parameters
    x0 = 2 * np.pi * np.random.random(num_params)
    circuit = circuit.assign_parameters(dict(zip(circuit.parameters, x0)))
    return circuit

# def Epsilon(rho, kraus_operators):
#     # K = K_noise = [\sqrt(p) I @ I, \sqrt(1-p) Z @ Z]
#     # see Eq. 1 Ref. [1]
#     return sum(K @ rho.data @ np.transpose(np.conjugate(K)) for K in kraus_operators)

# def Epsilon2(rho, unitary_matrix):
#     # K = K_noise = [\sqrt(p) I @ I, \sqrt(1-p) Z @ Z]
#     # see Eq. 1 Ref. [1]
#     return (np.transpose(np.conjugate(unitary_matrix)) @ rho.data @ unitary_matrix)

def calculate_rho_3 (rho, unitary_matrix, kraus_operators):
    '''
        sum(K @ U @ rho @ U(dagger) @ K(dagger)) //rho'' = rho
    '''
    rho_2 = unitary_matrix  @ rho @ np.transpose(np.conjugate(unitary_matrix))
    rho_3 = sum(K @ rho_2 @ np.transpose(np.conjugate(K)) for K in kraus_operators)
    return rho_3

def create_kraus_operators(unitary_matrix):
    '''
        Create a set of Kraus Operators from the input unitary matrix, using QR decomposition
    '''
    kraus_ops = []
    Q, R = qr(unitary_matrix)

    #Q: a 2^N x 2^N matrix, N is the number of qubits
    for q in Q:
        q = np.expand_dims(q, 1)
        kraus_ops.append(q @ np.transpose(np.conjugate(q)))
    return tf.convert_to_tensor(kraus_ops)

def compilation_trace_fidelity(rho, sigma):
    """Calculating the fidelity metric

    Args:
        - rho (DensityMatrix): first density matrix
        - sigma (DensityMatrix): second density matrix

    Returns:
        - float: trace metric has value from 0 to 1
    """
    rho_2 = tf.linalg.sqrtm((tf.linalg.sqrtm(rho)) @ (rho))

    # Cast to a supported type
    real_part = tf.math.real(rho_2)
    imaginary_part = tf.math.imag(rho_2)

    # Check for NaNs in both real and imaginary parts
    contains_nan_real = tf.reduce_any(tf.math.is_nan(real_part))
    contains_nan_imag = tf.reduce_any(tf.math.is_nan(imaginary_part))

    contains_nan = contains_nan_real or contains_nan_imag

    if contains_nan == True:
        rho_2 = rho

    return tf.linalg.trace(
            rho_2
            @ (tf.linalg.sqrtm(sigma))
        )

def frobenius_norm(rho, sigma):
    """
    Compute the Frobenius norm between two matrices.

    Parameters:
    rho (numpy.ndarray): The first matrix.
    sigma (numpy.ndarray): The second matrix.

    Returns:
    float: The Frobenius norm between rho and rho_prime.
    """

    # Ensure the matrices are TensorFlow tensors
    rho = tf.convert_to_tensor(rho, dtype=tf.complex128)
    sigma = tf.convert_to_tensor(sigma, dtype=tf.complex128)

    # Compute the difference between the matrices
    diff = rho - sigma

    # Compute the Frobenius norm
    #norm = tf.linalg.normalize(diff, ord='fro')
    norm = tf.sqrt(tf.reduce_sum(tf.square(diff)))
    return norm

#cost func to compare 2 given rhos
def cost(rho, rho_3):
    return tf.square(frobenius_norm(rho, rho_3))


In [144]:
n = 3

#Create a ansatz V with N qubits
circuit = V(n)

print(circuit)

     ┌────────────┐   ┌────────────┐   
q_0: ┤ Ry(4.8083) ├─■─┤ Ry(4.9235) ├─■─
     ├────────────┤ │ └────────────┘ │ 
q_1: ┤ Ry(4.1907) ├─■────────────────┼─
     ├────────────┤                  │ 
q_2: ┤ Ry(3.7129) ├──────────────────■─
     └────────────┘                    


In [145]:
# Get the unitary operator corresponding to the circuit
unitary_op = Operator(circuit)

# Get the unitary matrix
unitary_matrix = unitary_op.data

print (unitary_matrix) #2^N x 2^N

[[ 0.0215799 +0.j  0.13945682+0.j  0.03729533+0.j  0.24101535+0.j
   0.07348435+0.j  0.47488131+0.j  0.12699885+0.j  0.82071057+0.j]
 [-0.13945682+0.j  0.0215799 +0.j -0.24101535+0.j  0.03729533+0.j
  -0.47488131+0.j  0.07348435+0.j -0.82071057+0.j  0.12699885+0.j]
 [-0.24347942+0.j -0.01403971+0.j  0.14088258+0.j  0.00812369+0.j
  -0.82910127+0.j -0.04780831+0.j  0.47973635+0.j  0.02766295+0.j]
 [-0.01403971+0.j  0.24347942+0.j  0.00812369+0.j -0.14088258+0.j
  -0.04780831+0.j  0.82910127+0.j  0.02766295+0.j -0.47973635+0.j]
 [-0.07348435+0.j -0.47488131+0.j -0.12699885+0.j -0.82071057+0.j
   0.0215799 +0.j  0.13945682+0.j  0.03729533+0.j  0.24101535+0.j]
 [-0.47488131+0.j  0.07348435+0.j -0.82071057+0.j  0.12699885+0.j
   0.13945682+0.j -0.0215799 +0.j  0.24101535+0.j -0.03729533+0.j]
 [ 0.82910127+0.j  0.04780831+0.j -0.47973635+0.j -0.02766295+0.j
  -0.24347942+0.j -0.01403971+0.j  0.14088258+0.j  0.00812369+0.j]
 [-0.04780831+0.j  0.82910127+0.j  0.02766295+0.j -0.47973635+0.j
   

In [146]:
# Initialize a set of Kraus Operators from the given circuit
kraus_operators = create_kraus_operators(unitary_matrix=unitary_matrix)

#print(sum(K @ np.transpose(np.conjugate(K)) for K in KrausOperators))
print(unitary_matrix @ np.transpose(np.conjugate(unitary_matrix)))


[[ 1.00000000e+00+0.j  4.59240208e-18+0.j -4.11521248e-18+0.j
   8.44090189e-17+0.j  7.24042751e-18+0.j  1.75597570e-18+0.j
  -5.19375814e-18+0.j  4.25532603e-17+0.j]
 [ 4.59240208e-18+0.j  1.00000000e+00+0.j  6.56493390e-17+0.j
   4.15732631e-18+0.j -2.32005826e-18+0.j -3.46147373e-17+0.j
  -3.94136922e-17+0.j -8.83899365e-19+0.j]
 [-4.11521248e-18+0.j  6.56493390e-17+0.j  1.00000000e+00+0.j
  -2.01476421e-18+0.j  7.33305864e-18+0.j -3.72747235e-17+0.j
   3.00560429e-17+0.j -1.20482614e-17+0.j]
 [ 8.44090189e-17+0.j  4.15732631e-18+0.j -2.01476421e-18+0.j
   1.00000000e+00+0.j  4.80307425e-17+0.j  6.45492940e-18+0.j
  -1.31464347e-17+0.j  3.95738480e-17+0.j]
 [ 7.24042751e-18+0.j -2.32005826e-18+0.j  7.33305864e-18+0.j
   4.80307425e-17+0.j  1.00000000e+00+0.j -4.22001353e-18+0.j
  -3.06952367e-18+0.j -8.21335922e-17+0.j]
 [ 1.75597570e-18+0.j -3.46147373e-17+0.j -3.72747235e-17+0.j
   6.45492940e-18+0.j -4.22001353e-18+0.j  1.00000000e+00+0.j
  -1.49091817e-17+0.j  1.76424353e-17+0.j

In [147]:
#Initialize rho
rho = DensityMatrix.from_label('0' * n)
rho = tf.convert_to_tensor(rho)

#calculate rho_3
rho_3 = calculate_rho_3(rho=rho, kraus_operators=kraus_operators, unitary_matrix=unitary_matrix) 

#print(rho)
#print(rho3)


In [148]:
print(cost(rho=rho, rho_3=rho_3))

tf.Tensor((1.1253000836347191+0j), shape=(), dtype=complex128)


In [149]:
# Step 3: Implement the following function
# state_need_tomogaphy = ...
# rho = np.conjugate(np.transpose(state_need_tomogaphy)) @ state_need_tomogaphy   # density matrix
# rho' = Delta(rho)
# compiler = qoop.qcompilation.QuantumCompilation(U = rho', V = V())
# compiler.fit()
# compiler.plot()
# see fidelities versus iteration

In [150]:
#Auto Diff
'''
    rho: The initial rho
    unitary_matrix: randomly generated circle in initialization step
    kraus_operators: set of kraus operators
    n: number of qubits
    alpha: learning rate (?)'''
def calculate_derivative(rho, unitary_matrix, kraus_operators, n, alpha=0.1):
    tensorKraus = tf.Variable(kraus_operators)
    with tf.GradientTape() as tape:
        y = calculate_rho_3(rho, unitary_matrix, tensorKraus)
        f = cost(rho, y) 
    
    # Get the gradient of y with respect to x
    c = tape.gradient(f, tensorKraus)

    # Reshape c to kN * N matrix
    c = tf.reshape(c, (2**n * 2**n,2**n))

    # Reshape kraus_operators to kN * N matrix
    kraus_operators = tf.reshape(kraus_operators, (2**n * 2**n,2**n))
    
    # Compute the projection term
    proj = c - kraus_operators @ (np.transpose(np.conjugate(c)) @ kraus_operators  + np.transpose(np.conjugate(kraus_operators)) @ c) / 2

    # Update the Kraus operators
    updated_kraus_operators = kraus_operators - alpha * proj
    
    # Return the updated Kraus operators
    return updated_kraus_operators



In [151]:
kraus_operators_copy = tf.identity(kraus_operators)

In [152]:
# try looping manually
for i in range (0, 1000):
    _cost = cost(rho, calculate_rho_3(rho, unitary_matrix, kraus_operators_copy))

    #Update Kraus Operators
    kraus_operators_copy=calculate_derivative(rho, unitary_matrix, kraus_operators_copy, n)

    #Reshape
    kraus_operators_copy = tf.reshape(kraus_operators_copy, (2**n,2**n,2**n))
    
    #print('K(t)K\n',sum(K @ np.transpose(np.conjugate(K)) for K in KrausOperatorsTry)) # I

    print (_cost)

tf.Tensor((1.1253000836347191+0j), shape=(), dtype=complex128)
tf.Tensor((0.9623579883918605+0j), shape=(), dtype=complex128)
tf.Tensor((0.7510872478396287+0j), shape=(), dtype=complex128)
tf.Tensor((0.5238597477621084+0j), shape=(), dtype=complex128)
tf.Tensor((0.33262596130935346+0j), shape=(), dtype=complex128)
tf.Tensor((0.203297003690136+0j), shape=(), dtype=complex128)
tf.Tensor((0.12619740536264581+0j), shape=(), dtype=complex128)
tf.Tensor((0.08189379686112842+0j), shape=(), dtype=complex128)
tf.Tensor((0.05603925127633682+0j), shape=(), dtype=complex128)
tf.Tensor((0.04034527898244569+0j), shape=(), dtype=complex128)
tf.Tensor((0.03035014740295026+0j), shape=(), dtype=complex128)
tf.Tensor((0.023668190427190164+0j), shape=(), dtype=complex128)
tf.Tensor((0.018997700909103174+0j), shape=(), dtype=complex128)
tf.Tensor((0.01560452258690098+0j), shape=(), dtype=complex128)
tf.Tensor((0.013058008974186746+0j), shape=(), dtype=complex128)
tf.Tensor((0.011094876254992263+0j), shape=

In [153]:
print(sum(K @ np.transpose(np.conjugate(K)) for K in kraus_operators_copy))
print(kraus_operators_copy)

tf.Tensor(
[[ 1.67935627e+00+0.j  2.83299850e-03+0.j -4.95644003e-02+0.j
  -3.15395879e-02+0.j -9.05069344e-02+0.j -5.72101748e-03+0.j
   4.55618887e-02+0.j  1.17093124e-02+0.j]
 [ 2.83299850e-03+0.j  9.90962828e-01+0.j  1.81787857e-02+0.j
   1.22971064e-04+0.j -1.37898790e-03+0.j -2.17113242e-02+0.j
  -1.31724971e-03+0.j -1.18722166e-02+0.j]
 [-4.95644003e-02+0.j  1.81787857e-02+0.j  7.95644156e-01+0.j
  -2.46933432e-02+0.j  9.08092439e-04+0.j -1.20009203e-03+0.j
   6.09800789e-03+0.j  3.89179353e-04+0.j]
 [-3.15395879e-02+0.j  1.22971064e-04+0.j -2.46933432e-02+0.j
   9.91028519e-01+0.j -1.59924536e-03+0.j -1.19543107e-02+0.j
   9.10266003e-04+0.j -7.97327021e-03+0.j]
 [-9.05069344e-02+0.j -1.37898790e-03+0.j  9.08092439e-04+0.j
  -1.59924536e-03+0.j  8.61939427e-01+0.j -7.71312085e-03+0.j
   4.38438555e-02+0.j  1.39340380e-02+0.j]
 [-5.72101748e-03+0.j -2.17113242e-02+0.j -1.20009203e-03+0.j
  -1.19543107e-02+0.j -7.71312085e-03+0.j  9.23520394e-01+0.j
   1.33175082e-02+0.j -3.69089

In [154]:
#Check the output of sum(K @ U @ rho @ U(dagger) @ K(dagger))  - suppose to be rho
rho_out_1 = calculate_rho_3(rho=rho, unitary_matrix=unitary_matrix, kraus_operators=kraus_operators_copy)

In [160]:
#Compare with rho
print(compilation_trace_fidelity(rho,rho_out_1))

tf.Tensor((0.9995953217898459+0j), shape=(), dtype=complex128)


In [156]:
print (rho_out_1)

tf.Tensor(
[[ 9.99190807e-01+0.j  6.40651010e-07+0.j -8.21723009e-06+0.j
  -2.19860538e-06+0.j -7.77800364e-06+0.j -7.97140444e-07+0.j
   4.17508237e-06+0.j  7.89220801e-07+0.j]
 [ 6.40651010e-07+0.j  1.14682798e-04+0.j -3.01692763e-05+0.j
  -4.10851662e-05+0.j  1.73222062e-05+0.j  2.10178736e-04+0.j
   4.23359122e-06+0.j  8.77613009e-05+0.j]
 [-8.21723009e-06+0.j -3.01692763e-05+0.j  4.34655494e-04+0.j
  -1.43089843e-05+0.j -1.09236909e-04+0.j  3.89451360e-06+0.j
   9.05465039e-05+0.j  1.93907984e-05+0.j]
 [-2.19860538e-06+0.j -4.10851662e-05+0.j -1.43089843e-05+0.j
   1.58838638e-04+0.j -2.90690836e-05+0.j  8.59619342e-05+0.j
   2.57528642e-05+0.j  1.12396566e-04+0.j]
 [-7.77800364e-06+0.j  1.73222062e-05+0.j -1.09236909e-04+0.j
  -2.90690836e-05+0.j  4.82647969e-04+0.j  2.54264828e-05+0.j
  -4.36629574e-05+0.j -2.98160348e-06+0.j]
 [-7.97140444e-07+0.j  2.10178736e-04+0.j  3.89451360e-06+0.j
   8.59619342e-05+0.j  2.54264828e-05+0.j  7.69206419e-04+0.j
  -1.56004984e-05+0.j  2.31858